#### 1. Extracting the loans data from the cloud

A db_utils.py file is created to contain the code that connects with the AWS RDS cloud engine using the database credentials, extracts the loans data from the cloud and then finally saves the data locally in .csv format. Credentials.yaml file (containing RDS database credentials) is stored locally and ignored by GitHub for security reasons.
<br>
<br>
<ins>*The Python code consists of three parts:*</ins>
 - **load_credentials_file() function.** The function when called returns the credentials information presented in a dictionary format.
 - **class RDSDatabaseConnector.** The class contains *create_engine()* method that creates an engine to connect to the remote dabatase and extract_RDS_data(engine) method that uses that engine to extract the loans data, formats it into Pandas DataFrame and returns it as 'data' variable (where the loans data is contained).
 - **save_to_csv() function.** The function when called the data extracted from the remote RDS database is saved locally in .csv format and named "loan_payments".</ul>
<br>
RDSDatabaseConnector is using the AWS RDS database credentials information, which is locally stored in credentials.yaml file (and ignored by GitHub), to connect with the cloud server and then extract the loans data information for temporary viewing. The *save_to_csv()* function, when called, saves all the loans data locally at the chosen location in order to have easier access to the read, modify and analyse and the data.

In [ ]:
import yaml

def load_credentials_file():
    with open('credentials.yaml', 'r') as f:
        cred = yaml.safe_load(f)
    return cred

credentials = load_credentials_file()

In [ ]:
from db_utils import RDSDatabaseConnector

db_connect = RDSDatabaseConnector(credentials)

engine = db_connect.create_engine()

df = db_connect.extract_RDS_data(engine)

print(df.head())

In [ ]:
def save_to_csv():
    df.to_csv("C:/Users/eveli/ai_core/EDA/loan_payments.csv")
    
save_to_csv()

### 2. Load the locally saved loans data.
<br>
The loans data is loaded and stored in a variable called df_raw (short for "raw dataframe").

In [ ]:
import pandas as pd

df_raw = pd.read_csv("C:/Users/eveli/ai_core/EDA/loan_payments.csv")

### 3. Importing classes necessary for the work with the loans dataframe. 

Four classes are imported from separate python files:
 - Class DataTransform: changes the data types of columns that have been assigned with the wrong data type as well as rounds the values in columns to two decimal places.
 - Class DataFrameInfo: provides useful information about the data frame for the purpose of familiarising with the data before the cleaning process. 
 - Class Plotter: provides visual insights of the data.
 - Class DataFrameTransform: contains various methods used for cleaning and transformation of the data frame (such as methods to deal with NaN values, to correct skewness of the data, removal or transformation of outliers and removal of correlated columns)

In [ ]:
from class_data_transform import DataTransform

from class_data_frame_info import DataFrameInfo

from class_plotter import Plotter

from class_data_frame_transform import DataFrameTransform

### 4. Extract insightful information from the data.

Class DataFrameInfo is created to generate useful information about the data which helps us to familiarise ourselves with it. 
<br>
There are seven methods in the class:
 - show_colm_dtypes()
 - show_colm_head()
 - show_colm_tail()
 - show_colm_stat_values()
 - show_categ_colm_values_count()
 - show_df_shape()
 - show_null_percentage()</ul>
<br>
<ins>The first method</ins> (*show_colm_dtypes()*) shown below lists the name of every column in the data frame (including the index column *"Unnamed"* created by pandas), the count of non-null values each column has and, most importantly, what data type each column is assigned. This information helps us to establish whether any of the columns were assigned with the wrong data type. 

In [ ]:
info = DataFrameInfo(df_raw)

info.show_colm_dtypes()

<ins>The second method</ins> (*show_colm_head()*) shows the values of the top 25 rows of each column (categorical data columns are excluded).

In [ ]:
info.show_colm_head()

<ins>The third method</ins> (*show_colm_tail()*) shows the values of the bottom 25 rows of each column (categorical data columns are excluded).

In [ ]:
info.show_colm_tail()

<ins>The fourth method</ins> (*show_colm_stat_values()*) generates statistical values such as mean, standard deviation, count of data entries, min & max values as well as 25/50/75 percentiles for each column (includes only the columns containing continuous values).  

In [ ]:
info.show_colm_stat_values() 

<ins>The fifth method</ins> (*show_categ_colm_values_count()*) shows the count of data entries for each value in categorical columns (the columns containing categorical data). 
<br>
This helps to group the data entries of each column into groups for better insight. 

In [ ]:
info.show_categ_colm_values_count() 

<ins>The sixth method</ins> (*show_df_shape()*) shows the shape of the data frame. In this case, the shape of our data frame is (54231, 44) meaning it contains 44 columns and 54231 rows. 

In [ ]:
info.show_df_shape()

<ins>The last (seventh) method</ins> (*show_null_percentage()*) shows the percentage of null values in each column. This information will be essential in finding out which columns contains null values and based on the percentage of null values, in choosing which cleaning method is more appropriate in dealing with the null values. 

In [ ]:
info.show_null_percentage()

### 5. Convert columns to the correct data format.
After using class DataFrameInfo to analyse the given raw data, it was noticed that some of the columns could be assigned with better data format. For this purpose class DataTransform was created. 
<br>
<br>
There are three methods in the class:

 - float_to_int()
 - object_to_datetime()
 - round_values()</ul>
<br>
<ins>The first method</ins> (*float_to_int()*) transforms "funded_amount_inv" and "annual_inc" columns from float ('float64') to integer ('int64') data type as the data in these columns only consists of integer values. There are other columns such as "funded_amount", "mths_since_last_delinq", mths_since_last_record", "collections_12_mths_ex_med" and "mths_since_last_major_derog" that could benefit with this exact transformation as well however since they contain Nan values, the transformation would be returned with ValueError error. This is because NaN values are not considered valid integers, and therefore cannot be converted to integers.
<br>
<br>
<ins>The second method</ins> (*object_to_datetime()*) transforms "issue_date", "earliest_credit_line", "last_payment_date", "next_payment_date" and "last_credit_pull_date" columns from string ('object') to datetime ('datetime64') data type as the data in these columns represents dates. 
<br>
<br>
<ins>The third method</ins> (*round_values()*) rounds the values in "total_rec_late_fee" and "collection_recovery_fee" columns by two decimal values as the values represents a monetary value which by convention is expressed only up to two decimal values (for example, 22.23 value meaning 22 pounds and 23 pence).
<br>
<br>
After all the methods are called, we use *show_colm_dtypes()* method from class DataFrameInfo to check whether the data types of the columns were successfully changed. 


In [ ]:
transform = DataTransform(df_raw)

columns_list = ["funded_amount_inv", "annual_inc"]
for column in columns_list:
    transform.float_to_int(column)

col_list = ["issue_date", "earliest_credit_line", "last_payment_date", "next_payment_date", "last_credit_pull_date"]
for column in col_list:
    transform.object_to_datetime(column)

colmn_list = ["total_rec_late_fee", "collection_recovery_fee"]
for column in colmn_list:
    transform.round_values(column)

info.show_colm_dtypes()


### 6. Remove or impute missing values in the data.

Missing values in the dataset are a common phenomenon (especially in a large dataset like the current one) and it can be caused by errors in data collections, respondents not providing information or/and observations not being recorded. It is essential to deal with missing values (expressed in the dataset as 'Nan') as leaving them would make it hard to analyse the data (due to reasons such as creating biased results or many machine algoriths not being able to parse null values).

We will be using the class Plotter to visualise the Nan values and in this way identify in which columns they are present as well as class DataFrameTransform which will be dealing with the missing values by either imputing or removing them. 

By calling *null_values* (a method from class Plotter) we can see the percentage of Nan values that consists in each column and then a graphical representation of this. 

In [ ]:
visuals = Plotter(df_raw)

null_values = visuals.show_null_values()

This shows us that 11 columns contain Nan values:

 - "funded_amount" (5.5%)
 - "term" (8.8%)
 - "int_rate" (9.5%)
 - "employment_length" (3.9%)
 - "mths_since_last_delinq" (57.2%)
 - "mths_since_last_record" (88.6%)
 - "last_payment_date" (0.13%)
 - "next_payment_date" (60.1%)
 - "last_credit_pull_date" (0.01%)
 - "collections_12_mths_ex_med" (0.09%)
 - "mths_since_last_major_derog" (86.2%)</ul>
<br> 
Columns containing continuous data such as "funded_amount" and "int_rate" have just under 10% of Nan values. This number is too large to apply dropping rows (that contains Nan values) method and too small to just drop the entire columns, which means the Nan values will need to be imputed instead. The Nan values can be imputed with mean if the data is normally distributed or with median if the data is skewed. We will call the class Plotter *test_normal_distribution()* method to test the normal distribution and see the graphical illustration of the distribution for these two columns

In [ ]:
nd_columns = ["funded_amount", "int_rate"]
for colm in nd_columns:
    visuals.test_normal_distribution(colm)

From the graphs above we can see the data in "funded_amount" and "int_rate" columns is skewed therefore we will impute the Nan values with the median rather than the mean. This can be done by calling *impute_with_median()* method from class DataFrameTransform.

In [ ]:
df_transform = DataFrameTransform(df_raw)

median_columns = ["funded_amount", "int_rate"]
for column in median_columns:
    df_transform.impute_with_median(column)

The output produced by the method above states that both "funded_amount" and "int_rate" columns contain 54231 of non-null values which is the total number of rows contained in each column. This proves that Nan values in these columns have been dealt with. 
<br>
<br>
Since "term" and "employment_length" are categorical columns and also have relatively high percentage of Nan values (8.8% and 3.9% respectively), we should not drop the rows with Nan values (as this will make us lose quite significant amount of data) but instead impute them with mode (mean or median should only be used for continuous data). This can be done by calling *impute_with_mode()* method from class DataFrameTransform.

In [ ]:
mode_columns = ["term", "employment_length"]
for mod_column in mode_columns:
    df_transform.impute_with_mode(mod_column)

Once again the output produced by the method above states that "term" and "employment_length" columns contain 54231 of non-null values. This proves that Nan values in these columns have been dealt with. 
<br>

The "last_payment_date", "last_credit_pull_date" and "collections_12_mths_ex_med" columns contain less or equal to 0.1% of Nan values. As the percentage is so small, it is safe to drop the rows containing Nan values for these columns. This can be done by calling *drop_rows()* method from class DataFrameTransform. 

In [ ]:
row_drop_columns = ["last_payment_date", "last_credit_pull_date", "collections_12_mths_ex_med"]
for row_column in row_drop_columns:
    df_transform.drop_rows(row_column)

Since in the above method, the rows containing Nan values had to be dropped, we are observing the decrease in total row number/non-null values. We can see that 73 rows containing Nan values were dropped in "last_payment_date" (54231-54158=73), 7 rows were dropped in "last_credit_pull_date" (54158-54151=7) and 51 rows were dropped in "collections_12_mths_ex_med" (54151-54100=51). This should leave us with a total of 54100 rows in the dataset. We can check this by calling the *show_df_shape()* method from class DataFrameInfo which shows the current shape of the data frame. 


In [ ]:
info.show_df_shape()

Finally, the remaining "mths_since_last_delinq", "mths_since_last_record", "next_payment_date" and "mths_since_last_major_derog" columns contain more than 50% of Nan values which means we cannot impute the Nan values (as this would mean filling more than half of the data with imputed values) or drop the rows containing Nan values (as this would mean dropping more than half of the entire data frame). In this case the entire columns will be dropped. This can be done by calling *drop_columns()* method from class DataFrameTransform. 

In [ ]:
columns_dropped = ["mths_since_last_delinq", "mths_since_last_record", "next_payment_date", "mths_since_last_major_derog"]
for column in columns_dropped:
    df_transform.drop_columns(column)

We can check this by calling *show_colm_dtypes()* method from class DataFrameInfo which shows the names of all the columns in the data frame. In that list we will not see the columns that we just dropped. We can also check this by calling the *show_df_shape()* which shows the shape of the data frame. The number of columns should be decreased by 4 (from 44 columns to 40).

In [ ]:
info.show_colm_dtypes()

info.show_df_shape()

As we have finished with dealing with Nan values, we can call *show_null_values()* method from class Plotter to visually check the work we did. The graph now should not contain any Nan values as well as the percentage of Nan values for each column should say 0%.

In [ ]:
visuals.show_null_values()

Since none of the columns contain null values anymore, we can finish changing the data types (from float ('float64') to integer ('int64')) of columns which we could not change before due to them containing Nan values. Since we have dropped "mths_since_last_delinq", "mths_since_last_record" and "mths_since_last_major_derog", we are left with "funded_amount" and "collections_12_mths_ex_med" columns that still needs data type changing. This can be done by once again calling *float_to_int()* method from class DataTransform. Afterwards, we can check the result by calling *show_colm_dtypes()* method from class DataFrameInfo. 

In [ ]:
columns_list = ["funded_amount", "collections_12_mths_ex_med"]
for column in columns_list:
    transform.float_to_int(column)
    
info.show_colm_dtypes()

### 7. Make a copy of the data frame.
<br>
<br>
Let us create a copy of a data frame to compare the results later in the project. 

In [ ]:
m4_df = df_raw.copy()

### 8. Perform transformations on skewed data. 
<br>
<br>
Data that is not symmetrically distributed around the mean (a.k.a. skewed data) can influence the interpretation of the data as many statistical techniques assume that the data is normally distributed. Once data is found to be skewed, data transformations may need to be performed to fix the skewness. 

The *show_skewness()* method from class Plotter will be used to calculate and illustrate the level of skewness in each column containing continuous data. 
<br>
<br>
These are the thresholds to determine the skewness of data:
 - between -0.5 and 0.5: the distribution of the data is **approximately symmetric**.
 - between -1 and -0.5 or between 0.5 and 1: the distribution of the data is **moderately skewed**. 
 - less than -1 or greater than 1: the distribution of the data is **highly skewed**.</ul>

The positive values indicate a **positive (right) skew**. This means that the mean and the median of the data will be less than the mode.
<br>
<br>
The negative values indicate a **negative (left) skew**. This would indicate that the mean and the median of the data will be less than the mode. 



In [ ]:
skew_columns = ["loan_amount", "funded_amount", "funded_amount_inv", "int_rate", "instalment", "annual_inc", "dti", "delinq_2yrs", "inq_last_6mths", "open_accounts", "total_accounts", "out_prncp", "out_prncp_inv", "total_payment", "total_rec_prncp"]
for sk_colm in skew_columns:
    visuals.show_skewness(sk_colm)

#### For ease let us group the columns into three groups.

*Columns containing symmetric data:*
 - int_rate
 - dti</ul>

*Columns containing moderately skewed data:*
 - loan_amount
 - funded_amount
 - funded_amount_inv
 - instalment
 - total_accounts</ul>

*Columns containing highly skewed data:*
 - annual_inc
 - delinq_2yrs
 - inq_last_6mths
 - open_accounts
 - out_prncp
 - out_prncp_inv
 - total_payment
 - total_rec_prncp</ul>

It is interesting to note that all the columns containing skewed data are *positively skewed* (as all the skew values are positive).

In order to transform columns containing moderately and highly skewed data, three methods of transformation were used: **Square Root**, **Log** and **Yeo-Johnson** transformation. All three methods were tried on each column to test which method transforms the data closest to the symmetric distribution. As a result three methods *transform_skew_square_root()*, *transform_skew_log()* and *transform_skew_yeojohnson()* were added in class DataFrameTransform to transform the moderately and highly skewed columns using the transformation method best fit to each column. 

*Square Root Transformation was used on:*
 - loan_amount
 - funded_amount
 - funded_amount_inv
 - instalment
 - total_accounts
 - total_payment
 - total_rec_prncp</ul>

*Log Transformation was used on:*
 - annual_inc
 - open_accounts</ul>

*Yeo-Johnson was used on:*
 - delinq_2yrs
 - inq_last_6mths
 - out_prncp
 - out_prncp_inv</ul>


In [ ]:
sqr_columns = ["loan_amount", "funded_amount", "funded_amount_inv", "instalment", "total_accounts", "total_payment", "total_rec_prncp"]
for clm in sqr_columns:
    df_transform.transform_skew_square_root(clm)

log_columns = ["annual_inc", "open_accounts"]
for colm in log_columns:
    df_transform.transform_skew_log(colm)

yeo_columns = ["delinq_2yrs", "inq_last_6mths", "out_prncp", "out_prncp_inv"]
for colmn in yeo_columns:
    df_transform.transform_skew_yeojohnson(colmn)

After all the transformations, the *show_skewness()* method from class Plotter is called again to observe the outcome.

In [ ]:
corrected_columns = ["loan_amount", "funded_amount", "funded_amount_inv", "instalment", "annual_inc", "delinq_2yrs", "inq_last_6mths", "open_accounts", "total_accounts", "out_prncp", "out_prncp_inv", "total_payment", "total_rec_prncp"]
for colm in corrected_columns:
    visuals.show_skewness(colm)

Most of the columns with skewed data were successfully transformed to have approximately symmetric data distribution except for "delinq_2yrs" column which still has highly skewed data (skewness was brought down from 5.32 to 1.87) as well as "out_prncp" and "out_prncp_inv" columns which still have moderately skewed data (for both the skewness was brought down from 2.35 to 0.53 (however the number is very close to the range of where data would be considered approximately symmetric)). This simply indicates that there might be some outliers present that needs to be dealt with.
<br>
<br>
### 9. Remove outliers from the data. 
<br>
<br>
Outliers are data points that differ considerably from the majority of the other observations in the dataset. In other words, outliers are values in the data set that are very large or small compared to the majority of the values in the data set. Such values can negatively affect the statistical analysis (such as showing higher or lower mean and mode of the data) and the training process of a machine learning algorithm resulting in lower accuracy. Therefore it is important to deal with the outliers in the data. 
<br>

In order to spot the outliers in categorical data, we can call *show_disc_prob_distr()* method from class Plotter which shows the value count and Discrete Probability Distribution of selected columns.

In [ ]:
dpd_columns = ["term", "grade", "sub_grade", "employment_length", "home_ownership", "verification_status", "loan_status", "payment_plan", "purpose", "policy_code", "application_type"]
for clmn in dpd_columns:
    show_dpd = visuals.show_disc_prob_distr(clmn)

The "home_ownership" column has a single value "NONE" which can be considered as an outlier. We can deal with it by replacing the "NONE" value to "OTHER" and therefore adding the value to other 63 "OTHER" values.
<br>
<br>
The "loan_status" column has several outliers (353 values of "Does not meet the credit policy. Status:Charged Off" and 966 values of "Does not meet the credit policy. Status:Fully Paid") which like in "home_ownership" column case can be assigned to a similar group (to 5500 values of "Charged Off" and to 27011 values of "Fully Paid, respectively).
<br>
<br>
The "payment_plan" column has a clear outlier which is 1 value of "y" while the rest of the 54099 values are "n". As this outlier is extremely small in comparison to the rest of the data, it can dealt with by removing the a single row containing it. This will leave us with 54099 rows remaining in our data frame. 
<br>
<br>
Finally, while "verification_status" column does not have any visibly obvious outliers in the illustration, however we can group the values in the column in a more organised way. Logically, it does not make sense to have three groups of values called "Source Verified", "Verified" and "Not Verified" when "Source Verified" basically means the same thing as "Verified". So, all the "Source Verified" values will be replaced into "Verified" in order to group the values into two categories ("Verified" and "Not Verified").

We can achieve all this by calling *categ_data_outliers_transform()* method from class DataFrameTransform.

In [ ]:
df_transform.categ_data_outliers_transform()

In order to check that the transformation was successful we can call the *show_disc_prob_distr()* method from class Plotter again while only specifying the names of the columns where transformations were made. 

In [ ]:
dpd_columns = ["home_ownership", "verification_status", "loan_status", "payment_plan"]
for clmn in dpd_columns:
    show_dpd = visuals.show_disc_prob_distr(clmn)

Now let us move on to the outliers in the columns containing continuous data. By calling *show_cont_data_outliers()* method from class Plotter we can illustrate the outliers. 

In [ ]:
Cont_data_columns = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'instalment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_accounts', 'total_accounts', 'out_prncp', 'out_prncp_inv', 'total_payment', 'total_payment_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_payment_amount', 'collections_12_mths_ex_med']
for colm in Cont_data_columns:
    show_cont_outliers = visuals.show_cont_data_outliers(colm)

From the illustrations above, we can conclude that columns "loan_amount", "funded_amount", "out_prncp" and "out_prncp_inv" do not have obvious outliers therefore these columns do not require outliers' removal. The "collections_12_mths_ex_med" visually seems to have four groups of outliers, however as it's majority of values are 0, we should not rush to remove the outliers. The *value_counts()* method should provide us the insight of the groups of values that the column contains. 

In [ ]:
df_raw["collections_12_mths_ex_med"].value_counts()

As the majority of values in "collections_12_mths_ex_med" are 0, the other 4 unique values (1.0, 2.0, 3.0.,4.0) are considered outliers. If we would decide to remove the outliers or replace them with a median or mean (which would be 0 again), the data would be lost as all the values would just be 0. As a result, we will not deal with the outliers in this column.  
<br>
<br>
The columns named "funded_amount_inv", delinq_2yrs", "total_rec_prncp", "int_rate", "instalment", "dti" and "total payment" have relatively small outliers which are located either above (indicating extremely high numbers are the outliers) or below (indicating extremely low numbers are the outliers) the box plot illustration. Due to the relatively small number of outliers, these outliers can be dealt with by simply eliminating them. We can do this by calling *iqr_outliers_removal()* method from class DataFrameTransform. 

In [ ]:
iqr_columns = ["funded_amount_inv", "delinq_2yrs", "total_rec_prncp", "int_rate", "instalment", "dti", "total_payment"]
for iqr_colm in iqr_columns:
    df_transform.iqr_outliers_removal(iqr_colm)

As we are eliminating the rows containing the outliers, we can check whether the outlier removal procedure worked by checking the length of the data frame before the outlier removal and after. As the length of the data frame have decreased from 54099 to 52088, this proves that outliers were successfully removed. 
<br>
<br>
Columns such as "annual_inc", "open_accounts" and "total_accounts" have quite a significant amount of outliers from both sides of the box plot. The outlier removal procedure (like the one used before) cannot be applied here as in this case quite a significant amount of rows would be lost from the data frame. Instead we can use a method where we set the flooring (the minimum value) and capping (the maximum value) for the dataset. This means that we would replace the data values that are below the 10th percentile with the 10th percentile values as well as replace the data values that are above the 90th percentile with 90th percentile values. This can be done by calling *flooring_capping_outliers_removal()* method from class DataFrameTransform. 

In [ ]:
floor_cap_columns = ["annual_inc", "open_accounts", "total_accounts"]
print(f'The length of data frame BEFORE capping and flooring of outliers: {len(df_raw)}\n')
for fl_cp_column in floor_cap_columns:
    df_transform.flooring_capping_outliers_transform(fl_cp_column)
print(f'The length of data frame AFTER the capping and flooring of outliers: {len(df_raw)}')

The identical length of data frame before and after the outliers transformation confirms that we only intended here to replace the outliers rather than remove them, therefore the length of the data frame should not change. Once the flooring and capping values are applied, these values should also be the new minimum and maximum values of each column. If that is the case, this means the transformation was successful. 
<br>
<br>
The remaining columns "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_payment_amount" and "total_payment_inv" contains very large amount of outliers above the box plot. Due to the large number of outliers there, it would not make sense to either remove or replace them because in both cases a large percentage of data values would be removed or changed, which can influence the interpretation of the data later on. As a result, we will not deal with the outliers for these columns. 
<br>
<br>
By calling the *show_cont_data_outliers()* method again and selecting only the column names in which the transformation took place, we should be able to see a drastically reduced amount of outliers present.

In [ ]:
Cont_data_columns = ["funded_amount_inv", "delinq_2yrs", "total_rec_prncp", "int_rate", "instalment", "dti", "total_payment", "annual_inc", "open_accounts", "total_accounts"]
for colm in Cont_data_columns:
    show_cont_outliers = visuals.show_cont_data_outliers(colm)

### 10. Dropping overly correlated columns. 
<br>
<br>
**Collinearity** occurs when two independent variables are heavily related to each other to extend where one of the independent variables can be predicted from the others with a very high degree of accuracy. **Multicollinearity** occurs when more than two independent variables are heavily related to each other. 
<br>
As collinearity occurs over the linear relationship between variables, we can call *show_correlation_cont_data()* method from Plotter which uses a correlation heatmap to illustrate the correlation between independent variables in our the data frame. 

In [ ]:
columns_list = ["loan_amount", "funded_amount", "funded_amount_inv", "int_rate", "instalment", "annual_inc", "dti", "delinq_2yrs", "inq_last_6mths", "open_accounts", "total_accounts", "out_prncp", "out_prncp_inv", "total_payment", "total_payment_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee","recoveries", "collection_recovery_fee", "last_payment_amount", "collections_12_mths_ex_med"]
visuals.show_correlation_cont_data(columns_list)

There is no strict rule for how much collinearity is too much, however a rule of thumb of anything above 0.9 is often used. After analysing the correlation heatmap, we can summarise the columns that have high correlation with other columns:
<br>
 - **"loan_amount"** is correlated with:&ensp; *"funded_amount"(0.966)*,&emsp;&emsp;&ensp; *"funded_amount_inv"(0.963)*,&emsp; *"instalment"(0.957)*.
 - **"funded_amount"** with:&emsp;&emsp;&emsp;&emsp;&emsp;*"funded_amount_inv"(0.942)*,&emsp; *"instalment"(0.937)*.
 - **"funded_amount_inv"** with:&emsp;&emsp;&emsp;*"instalment"*.
 - **"out_prncp"** with: &emsp; &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;*"out_prncp_inv"(0.999)*
 - **"total_payment"** with:  &emsp; &emsp;&emsp;&emsp;&emsp;*"total_payment_inv"(0.960)*,&emsp;&ensp; *"total_rec_prncp"(0.957)*.
 - **"total_payment_inv"** with:  &emsp;&emsp;&emsp;&ensp;*"total_rec_prncp"(0.915)*.</ul>
<br>
Note: In above when I mention, for example, a column_one is correlated with column_two, it also means column_two is correlated with column_one. By assuming this, I can avoid repeating myself later by saying, for example, column_two is related with column_one. 
<br>
<br>
The first thing we can notice is that there is multicollinearity as *"instalment"* has correlation with three other columns ("loan_amount", "funded_amount" and "funded_amount_inv"), *"funded_amount_inv"* has correlation with two other columns ("loan_amount" and "funded_amount") and "total_rec_prncp" has correlation with two other columns as well ("total_payment" and "total_payment_inv"). As a result, it make sense to drop these columns first. 
<br>
<br>
Next, we can notice that  *"out_prncp"* and *"out_prncp_inv"* are nearly perfectly collinear (the correlation number is very close to 1). This means that with the correlation as strong as this, it would be more difficult to change the data in one of the columns without changing the data in the other one and this can complicate the interpretation of the data. Therefore one of the columns should be removed. In order to decided which column to remove, we should analyse what the values in each column really represent:
<br>
<br>
<ins>out_prncp:</ins> Remaining outstanding principal(remaining amount of the original loan + any capitalized interest) for total amount funded.
<br>
<br>
<ins>out_prncp_inv:</ins> Remaining outstanding principal for portion of total amount funded by investors. 
<br>
<br>So "out_prncp" displays the remaining amount of the original loan for total amount funded (funded by anyone, therefore, a more general case) while "out_prncp_inv" displays the remaining amount of the original loan for portion of total amount, that is funded by investors (a more specific case that involves 'investors' as a specific group rather than 'anyone'). In this case it could make sense to drop the "out_prncp_inv" column in order to leave us with a more general case of "out_prncp" column data which could be more useful for later analysis. 
<br>
<br>
Finally, we have "loan_amount" column correlating with "funded_amount" and then "total_payment" correlating with "total_payment_inv". However, here we need to remember that we should not be dropping too many correlated columns as then, in some situations, we may be subjecting our data to omitted variable bias which means that by deleting the correlated column we can lose important features from that column that are needed for effective analysis of certain topics. 
<br>
<br>
A *drop_columns()* method from class DataFrameTransform can be used to drop the columns and then we can call *show_df_shape()* method from class DataFrameInfo to check the shape of the data frame before and after the removal of columns to confirm that the removal is completed. 

In [ ]:
info.show_df_shape()

columns_dropped = ["instalment", "funded_amount_inv", "total_rec_prncp", "out_prncp_inv"]
for column in columns_dropped:
    df_transform.drop_columns(column)
    
info.show_df_shape()

We can confirm that the column removal is complete by observing the number of columns decreasing from 40 to 36 columns. 